In [1]:
import openpyxl

In [2]:
%run MyFunctions.ipynb

In [3]:
## Import data
filename = "ResearchInChildAndAd_DATA_2018-12-14_1531.xlsx"
workbook = openpyxl.load_workbook(filename)
sheet1 = workbook["ResearchInChildAndAd_DATA_2018-"]
sheet2 = workbook["Tasks"]

In [4]:
## Module 1: Make columns for new data
IGNORE = ['query', 'vs', 'r/o', 'rule out']
KEYWORDS_DICT = {'ADHD': ['attention', 'adhd', 'add'], 
                 'ASD': ['autis', 'asd'],
                 'Anxiety': ['anxiety', 'panic', 'phobi'], 
                 'Bipolar': ['bipolar', 'mani'],
                 'Depression': ['depress', 'MDD', 'PDD', 'mood', 'dysthymi'], 
                 'EOS': ['schizo'], 
                 'FASD': ['fasd', 'fetal', 'alcohol'], 
                 'OCD': ['obsess', 'compuls', 'ocd'], 
                 'ODD': ['opposit', 'defiant', 'odd'], 
                 'Parent-Child': ['parent'], 
                 'Personality': ['cluster', 'personality', 'histrio', 'borderline'], 
                 'Psychosis': ['psychosis', 'psychotic'],
                 'SUD': ['substance', 'SUD', 'use disorder'], 
                 'Trauma': ['trauma', 'adjust', 'ptsd'], 
                 'eating': ['eating', 'anorexi', 'bulimi', 'binge'], 
                 'neurodevelopmental': ['learning', 'delay', 'neurodevelopment', 'intellectual', 'conduct', 'cerebral', 'palsy', 'chromosom'],
                 'somatic': ['somati'], 
                 'suicide': ['suicid']}
# dont forget handle fads and NOT personality for EOS
# read file in instead
BINARY_HEADERS = []
for i in KEYWORDS_DICT.keys():
    BINARY_HEADERS.append(i)

## Add columns
add_column(sheet1, "main_addx")
for i in range(0, len(BINARY_HEADERS)):
    add_column(sheet1, "addx_{}".format(BINARY_HEADERS[i]))
    
add_column(sheet1, "main_dcdx")
for i in range(0, len(BINARY_HEADERS)):
    add_column(sheet1, "dcdx_{}".format(BINARY_HEADERS[i]))

In [5]:
## Create a dictionary of column names
ColNames = {}
Current  = 0
for COL in sheet1.iter_cols(1, sheet1.max_column):
    ColNames[COL[0].value] = Current
    Current += 1

In [7]:
## Set binary values to 0
set_binaries_to_zero(sheet1, "admission", ColNames)
set_binaries_to_zero(sheet1, "discharge", ColNames)

In [8]:
## Module 2: Fill in main diagnosis

## Admission
main_diagnosis(sheet1, "admission", ColNames)

****************
Main diagnosis full description: adjustment disorder with disturbance of conduct
Main illness: neurodevelopmental
Non-zero illnesses at record_id = 21296
****************
Main diagnosis full description: fasd
Main illness: FASD
Non-zero illnesses at record_id = 11238
****************
Main diagnosis full description: reactive attachment disorder
Main illness: 
Non-zero illnesses at record_id = 11250
****************
Main diagnosis full description: disinhibited social engagement disorder
Main illness: 
Non-zero illnesses at record_id = 11257
****************
Main diagnosis full description: unspecified depressive disorder (r/o mdd)
Main illness: 
Non-zero illnesses at record_id = 21284
****************
Main diagnosis full description: query unspecified psychosis versus bipolar disorder
Main illness: 
Non-zero illnesses at record_id = 11267
****************
Main diagnosis full description: situational crisis
Main illness: 
Non-zero illnesses at record_id = 11268
********

In [9]:
## Discharge 
main_diagnosis(sheet1, "discharge", ColNames)

****************
Main diagnosis full description: unspecified disruptive
Main illness: 
Non-zero illnesses at record_id = 21296
****************
Main diagnosis full description: fasd
Main illness: FASD
Non-zero illnesses at record_id = 11238
****************
Main diagnosis full description: disruptive behavioural disorder
Main illness: 
Non-zero illnesses at record_id = 21265
****************
Main diagnosis full description: fix
Main illness: 
Non-zero illnesses at record_id = 11244
****************
Main diagnosis full description: none specified
Main illness: 
Non-zero illnesses at record_id = 11245
****************
Main diagnosis full description: query gender dysphoria
Main illness: 
Non-zero illnesses at record_id = 11247
****************
Main diagnosis full description: fasd
Main illness: FASD
Non-zero illnesses at record_id = 11252
****************
Main diagnosis full description: disinhibited social engagement disorder
Main illness: 
Non-zero illnesses at record_id = 11257
*****

In [10]:
## Module 3: Read Diagnosis, break into substrings, fill in

## Admissions
one_hot_encoding_diagnoses(sheet1, 'admission', ColNames)

## Discharge
one_hot_encoding_diagnoses(sheet1, 'discharge', ColNames)

In [11]:
## Output file
workbook.save("output-{}.xlsx".format(getTime()))